In [1]:
import numpy as np
import pandas as pd
import math
import SALib
from SALib.sample import latin
import plotly.graph_objects as go

In [2]:
# Define fontsizes
lab_font_size = 20
tick_font_size = 18
anno_font_size = 16
fig_path = '../../output/paper3/'

In [3]:
# Define the function
# Parameters x are defined on [-1, 1]
def diff_power(x1, x2):
    """
    Parameters:
    x1, x2: np.ndarray, matrix of parameter values for the Gaussian peak integrand family (GPI).
    
    Return:
    y: np.ndarray, the output of SDP function.
    """
    y = math.exp(-25 * (x1 - 0.5)**2 - 4*(x2 - 0.5) ** 2)
    return y
# vetorize diff_power fucntion 
v_func = np.vectorize(diff_power)

## Create the contour plot

In [4]:
# Create meshgrid points
x = np.array(np.meshgrid(np.linspace(0, 1, 1000),
                   np.linspace(0, 1, 1000)))

# Fix parameters at certain points and calculate the related  results
# Fix x2 at 0.0, 0.8 respectively
fix_1 = 0.5
fix_2 = 0.9
# Calculate the unconditioned results
z = v_func(x[0], x[1])

In [5]:
fig = go.Figure(data =
    go.Contour(
        z = z,
        x = x[0][0],
        y = x[1][:,0],
        colorbar = dict(title='Values', # title here
            titleside='top',
            titlefont=dict(
            size=lab_font_size),
            tickfont = dict(size = tick_font_size)
            ),
        colorscale='sunset',
    contours=dict(
            coloring ='heatmap',
            showlabels = True, # show labels on contours
            labelfont = dict( # label font properties
                size = 12,
                color = 'white'),
            start=0,
            end=1,
            size=0.1)
            ))

fig.add_trace(go.Scatter(x=[0.405, 0.405, 0.595, 0.595, 0.405], y=[0.26, 0.74, 0.74, 0.26, 0.26], 
                         fill=None, line_color='royalblue',
                        line_width=1.5, mode='lines'))

# Add annotation and arrows describing box
fig.add_annotation(x=0.55, y=0.74,
            text="The bounding box of feasible set",
            showarrow=True,
            arrowhead=1,
            font = {"size": anno_font_size}
            )

# Add annotations to contour plot
fig.add_hline(y=fix_1, line_width=1.5, line_dash="dash", line_color="red", 
              annotation_text = r'$f(x_1, x_2=%.1f)$'%fix_1, annotation_font_size = anno_font_size)
fig.add_hline(y=fix_2, line_width=1.5, line_dash="dash", line_color="green", 
              annotation_text = r'$f(x_1, x_2=%.1f)$'%fix_2, annotation_font_size = anno_font_size)
fig.update_xaxes(title_text = r'$x_1$', title_font = {"size": lab_font_size}, tickfont=dict(size=tick_font_size))
fig.update_yaxes(title_text = r'$x_2$',title_font = {"size": lab_font_size},  tickfont=dict(size=tick_font_size))
fig.write_image(f'{fig_path}contour.png', format = 'png', scale=2)

## Create boxplot

In [6]:
# random sampling to over parameter space and then fix variable x2
problem = {'num_vars': 2,
          'names': ['x1', 'x2'],
          'bounds': [[0, 1], 
                    [0, 1]]
          }
param_values_full = latin.sample(problem, 10000)
x1_full = param_values_full[:, 0]
x2_full = param_values_full[:, 1]
z_uncon = v_func(x1_full, x2_full)

# Calculate the conditional values with x2 fixed.
group_labels = [r'$f(x_1, x_2)$', r'$f(x_1, x_2=%.1f)$'%(fix_1), r'$f(x_1, x_2=%.1f)$'%(fix_2)]
z_c1 = v_func(x1_full, fix_1)
z_c2 = v_func(x1_full, fix_2)

In [7]:
hist_data = [z_uncon, z_c1, z_c2]
fig3 = go.Figure()
for z_c, lab in zip(hist_data, group_labels):
    fig3.add_trace(go.Box(y = z_c, name = lab, width = 0.2))
fig3.update_yaxes(title_text = r'$f\ values$', title_font = {"size": lab_font_size}, tickfont={"size" : tick_font_size})
fig3.update_xaxes(tickfont={"size" : tick_font_size}, title_font = {"size": lab_font_size})
fig3.update_layout(showlegend=False)
fig3.write_image(f'{fig_path}boxplot.png', format = 'png', scale=1)

## Create histgram

In [8]:
import plotly.figure_factory as ff
hist_data = [z_c1, z_c2]
fig2 = ff.create_distplot(hist_data, group_labels[1:], show_hist = False, show_rug = False)
fig2.update_yaxes(title_text = 'Probability distribution function', title_font = {"size": lab_font_size})
fig2.update_xaxes(title_text=r'$f\ values$', title_font = {"size": lab_font_size})
fig2.update_layout(legend=dict(yanchor = "top",
                              y = 0.8,
                              xanchor = "left",
                              x = 0.45))
fig2.write_image(f'{fig_path}histgram.png', format = 'png', scale=2)

## Create scatter plot

In [9]:
# Scatter plot of x2 fixed at fix_1
hist_data = [z_uncon, z_c1, z_c2]
fig4 = go.Figure()
k = 0
for ydata, lab in zip(hist_data, group_labels):
    if k != 1:  
        fig4.add_trace(go.Scatter(x = x1_full, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.7}))
    else:
        fig4.add_trace(go.Scatter(x = x1_full, y = ydata, mode='markers', name=lab, 
                                  marker = {'size': 6, 'opacity' : 0.02}))
    k += 1
fig4.update_yaxes(title_text = r'$f\ values$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
fig4.update_xaxes(title_text = r'$x_1$', title_font = {"size": lab_font_size}, tickfont = {"size" : tick_font_size})
fig4.update_layout(legend=dict(yanchor = "top",
                              y = 0.95,
                              xanchor = "left",
                              x = 0.65, 
                              font = {"size":14}))
fig4.write_image(f'{fig_path}scatterplots.png', format = 'png', scale=2)